In [376]:
import csv
import pandas
import numpy as np
import operator
xtest = pandas.read_csv('bbb.test.csv', header = None)
xtrain = pandas.read_csv('bbb.train.csv',header = None)
ytest = pandas.read_csv('labels.test.csv',header = None)
ytrain = pandas.read_csv('labels.train.csv',header = None)
yytrain = ytrain.replace('nc', 1)
fytrain = yytrain.replace('c', 0)
yytest = ytest.replace('nc', 1)
fytest = yytest.replace('c', 0)

In [638]:
import plotly.graph_objects as go
def confusionM(truey, prey):
    v00, v01, v10, v11 = 0, 0, 0, 0
    for i in range(len(truey)):
        if truey[i] == 0:
            if prey[i] == 0:
                v00 += 1
            else:
                v10 += 1
        else:
            if prey[i] == 0:
                v01 += 1
            else:
                v11 += 1
    fig = go.Figure(data=[go.Table(
        header=dict(values=['Confusion Matrix', 'True c(0)', 'True nc(1)', 'Total'],
                    fill_color='paleturquoise',
                    align='left'),
        cells=dict(values=[['Predict c(c)', 'Predict nc(1)', 'Total'], [v00, v10, v00+v10], [v01, v11, v01+v11], [v00+v01, v10+v11, ' ']],
                   fill_color='lavender',
                   align='left'))])
    fig.show()

In [695]:
def confusionM(truey, prey):
    v00, v01, v10, v11 = 0, 0, 0, 0
    for i in range(len(truey)):
        if truey[i] == 0:
            if prey[i] == 0:
                v00 += 1
            else:
                v10 += 1
        else:
            if prey[i] == 0:
                v01 += 1
            else:
                v11 += 1
    print('Confusion Matrix|', 'True c(0)', 'True nc(1)', 'Total')
    print('  Predicc c(0)  |',  v00,'       ', v01,'       ', v00+v01)
    print('  Predicc nc(1) |',  v10,'      ', v11,'      ', v10+v11)
    print('  Total         |',  v10+v00,'      ', v11+v01,'      ', v10+v11+v00+v01)

In [696]:
def PredictFunc(x, y, beta):
    res = {}
    res['error'] = 0
    res['prey'] = []
    res['truey'] = []
    for i in range(x.shape[0]):
        temp = x.iloc[i,:]
        tempty = y.iloc[i,0]
        temppy = PredOneRow(temp, beta)
        res['prey'].append(temppy)
        res['truey'].append(tempty)
        if tempty != temppy: 
            res['error'] += 1
    res['accu'] = 1 - res['error'] / x.shape[0]
    res['mse'] = res['error'] / x.shape[0]
    return res

def PredOneRow(x, beta):
    x = np.hstack((np.ones((1)), x.T))
    pre = pi_val(np.dot(x, beta))
    #print(x)
    if pre > 0.5:
        return 1
    return 0

In [628]:
def pi_val(input):
    output = 1 / (1 + np.exp(-input))
    return output

In [701]:
# Logistic Regression
def GradientD(x, y, stepsize, iters):
    beta = np.random.random_sample((x.shape[1],1))
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = np.dot(x.T, y - yy)
        beta = beta - stepsize*grad
    return beta

def LR(x, y, stepsize, iters):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD(x, y, stepsize, iters)
    return beta

def OutputFuncLo(beta, res):
    print('Coefficient: ', beta.flatten(), '\n')
    print('Error Num:', res['error'], '  Accuracy:', res['accu'])

betaLo = LR(xtrain, fytrain, 0.01, 900)
resLo = PredictFunc(xtest, fytest, betaLo)
OutputFuncLo(betaLo, resLo)
confusionM(resLo['truey'], resLo['prey'])



Coefficient:  [880.27730384 629.62086362 278.6965209   33.99473139 163.51012048
 -19.3702708  227.22949538 -40.24356053 282.27222101 100.56816303
 285.15639493  63.52031082 250.01224294 -21.89095089 135.26536458
 -41.40744022 275.87015118 -54.6988015  214.1114175   -2.1403074
 125.50069343  91.86877214 105.46847693 -28.38724597 176.30939527
 -13.15671439 561.73610091 -51.21471834 151.00226278 -35.52731485
  84.22932478  46.93120564 123.5900486   82.629763  ] 

Error Num: 27   Accuracy: 0.619718309859155
Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 0         0         0
  Predicc nc(1) | 27       44       71
  Total         | 27       44       71


In [773]:
def GradientD3(x, y, stepsize, iters, lam3):
    #beta = np.ones((x.shape[1],1))
    beta = np.random.random_sample((x.shape[1],1)) # 34*1
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = -2*np.dot(x.T, y - yy)+ lam3 * beta
        #print(grad)
        beta = beta - stepsize*grad
    return beta

def RR(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD3(x, y, stepsize, iters, lam3)
    return beta

def cvkfold(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = RR(xtrain_train, ytrain_train, 0.01, 500, lam3)
            restemp = PredictFunc(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse
 
def usebestlam(lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = 0
    for i in range(len(lambdalist)):
        if lambdalist[i] <= up:
            up = lambdalist[i]
            index = i
    bestlam = lambdapotential[index]
    beta1 = RR(xtrain, fytrain, 0.01, 500, bestlam)
    res = PredictFunc(xtest, fytest, beta1)
    print('Accuracy: ', res['accu'], ' Best Lambda: ', bestlam)
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

lambdapotential = [0.03, 0.005, 0.1]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold(data, lambdapotential, 5)
print(lambdalist)
acc, lam = usebestlam(lambdapotential, xtrain, fytrain, xtest, fytest)


[[0.13571429]
 [0.13571429]
 [0.13928571]]
Accuracy:  0.8169014084507042  Best Lambda:  0.005
Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 15         1         16
  Predicc nc(1) | 12       43       55
  Total         | 27       44       71


In [936]:
def soft_threshold(beta, lam):
    t = 1
    for i in range(len(beta)):
        if beta[i] > t*lam:
            beta[i] -= t*lam
        elif beta[i] < -t*lam:
            beta[i] += t*lam
        else:
            beta[i] = 0
    return beta

def GradientD4(x, y, stepsize, iters, lam3):
    beta = np.random.random_sample((x.shape[1],1)) # 34*1
    beta1 = beta
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = -2*np.dot(x.T, y - yy)+ soft_threshold(beta1, lam3)
        #print(grad)
        beta1 = beta - (lam3*grad)/x.shape[0]
        beta = beta - (stepsize*grad)/x.shape[1]
    return beta

def LA(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD4(x, y, stepsize, iters, lam3)
    return beta


def cvkfold4(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = LA(xtrain_train, ytrain_train, 0.01, 100, lam3)
            restemp = PredictFunc(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse
 
def usebestlam4(lambdalist, lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = -1
    for i in range(len(lambdalist)):
        if lambdalist[i] <= up:
            up = lambdalist[i]
            index += 1
    bestlam = lambdapotential[index]
    beta1 = LA(xtrain, fytrain, 0.01, 300, bestlam)
    res = PredictFunc(xtest, fytest, beta1)
    print(beta1)
    print('Accuracy: ', res['accu'], ' Best Lambda: ', bestlam)
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

lambdapotential = [0.03, 0.01, 0.1, 0.2, 0.06,0.5, 0.8, 1, 2]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold4(data, lambdapotential, 5)
acc, lam = usebestlam4(lambdalist, lambdapotential, xtrain, fytrain, xtest, fytest)

[[-1.4196714 ]
 [ 0.48461385]
 [ 0.9957557 ]
 [ 0.22145031]
 [ 1.07250592]
 [ 0.7532544 ]
 [ 0.29770506]
 [ 0.96903852]
 [ 0.503525  ]
 [ 0.10041251]
 [-0.18280557]
 [ 0.04721599]
 [-0.02360138]
 [ 0.48849054]
 [ 0.24414897]
 [ 0.33533817]
 [-0.16415454]
 [ 0.44897832]
 [-0.18178728]
 [ 0.02527038]
 [ 0.21615285]
 [-0.8767832 ]
 [ 0.67809397]
 [ 0.20919794]
 [ 0.26334522]
 [ 0.12419342]
 [-1.27640233]
 [-0.15467067]
 [ 0.37455547]
 [ 0.19247259]
 [ 0.40013757]
 [-0.20049034]
 [ 0.15626468]
 [-0.79241769]]
Accuracy:  0.8309859154929577  Best Lambda:  0.5
Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 15         0         15
  Predicc nc(1) | 12       44       56
  Total         | 27       44       71


In [853]:
def soft_threshold(beta, lam):
    t = 0.5
    for i in range(len(beta)):
        if beta[i] > t*lam:
            beta[i] -= t*lam
        elif beta[i] < -t*lam:
            beta[i] += t*lam
        else:
            beta[i] = 0
    return beta

def GradientD4(x, y, stepsize, iters, lam3):
    beta = np.random.random_sample((x.shape[1],1)) # 34*1
    beta1 = beta
    for i in range(iters):
        yy = pi_val(np.dot(x, beta))
        grad = (np.dot(x.T, y - yy)+ soft_threshold(beta, lam)) /x.shape[0]
        #print(grad)
        beta1 = beta - (lam3*grad)
        beta = beta - (stepsize*grad)
    return beta

def LA(x, y, stepsize, iters, lam3):
    x = np.hstack((np.ones((x.shape[0], 1)), x))
    beta = GradientD4(x, y, stepsize, iters, lam3)
    return beta

def cvkfold4(data, lamrange, k):
    folds = np.array_split(data, k)
    tempmse = np.zeros((len(lamrange), 1))
    for i in range(k):
        train = folds.copy()
        test = folds[i]
        del train[i]
        
        train = pandas.concat(train, sort=False)
        #print(test.shape)
        xtrain_train = train.iloc[:, :33]
        #print(xtrain_train)
        ytrain_train = train.iloc[:, 33:]
        xtrain_test = test.iloc[:, :33]
        ytrain_test = test.iloc[:, 33:]
        
        for lam3 in lamrange:
            betatemp = LA(xtrain_train, ytrain_train,0.01, 100, lam3)
            restemp = PredictFunc1(xtrain_test, ytrain_test, betatemp)
            tempmse[lamrange.index(lam3)] += restemp['mse']
    return tempmse/k # average mse
 
def usebestlam4(lambdapotential, xtrain, fytrain, xtest, fytest):
    up = 100
    index = 0
    for i in range(len(lambdalist)):
        if lambdalist[i] <= up:
            up = lambdalist[i]
            index = i
    bestlam = lambdapotential[index]
    beta1 = LA(xtrain, fytrain, 0.01, 100, bestlam)
    res = PredictFunc1(xtest, fytest, beta1)
    print('Accuracy: ', res['accu'], ' Best Lambda: ', bestlam)
    confusionM(res['truey'], res['prey'])
    return res['accu'], bestlam

def PredictFunc1(x, y, beta):
    res = {}
    res['error'] = 0
    res['prey'] = []
    res['truey'] = []
    for i in range(x.shape[0]):
        temp = x.iloc[i,:]
        tempty = y.iloc[i,0]
        temppy = PredOneRow1(temp, beta)
        res['prey'].append(temppy)
        res['truey'].append(tempty)
        if tempty != temppy: 
            res['error'] += 1
    res['accu'] = 1 - res['error'] / x.shape[0]
    res['mse'] = res['error'] / x.shape[0]
    return res

def PredOneRow1(x, beta):
    x = np.hstack((np.ones((1)), x.T))
    pre = pi_val(np.dot(x, beta))
    #print(x)
    if pre > 0.5:
        return 1
    return 0


lambdapotential = [0.03, 0.01, 5, 10]
data = pandas.concat([xtrain, fytrain.reindex(xtrain.index)], axis=1)
lambdalist = cvkfold4(data, lambdapotential, 5)
print(lambdalist)
acc, lam = usebestlam4(lambdapotential, xtrain, fytrain, xtest, fytest)

[[0.68214286]
 [0.68214286]
 [0.68214286]
 [0.68214286]]
Accuracy:  0.3661971830985915  Best Lambda:  10
Confusion Matrix| True c(0) True nc(1) Total
  Predicc c(0)  | 26         44         70
  Predicc nc(1) | 1       0       1
  Total         | 27       44       71


In [667]:
from itertools import cycle
from copy import deepcopy
